# Review of Stop Loss and Drawdown Strategies

## Introduction
This project reviews various stop loss and drawdown strategies used in trading. The focus is to analyze their effectiveness in risk management and performance enhancement.

## Table of Contents
1. Introduction
2. Types of Stop Loss Strategies
3. Drawdown Analysis
4. Strategy Comparison
5. Results and Discussion
6. Conclusion
7. References

# Here is a review of both the drawdown and the stop-loss function:

# **Stop Loss**

This one I am particulary proud of. This is a vectorised stop loss function. Since I only read one *linear algebra* course at this time, you probably can condense it a bit more. Nevertheless, it is best that we go through it.



## Function Inputs and Parameters

- `volume_signals`: DataFrame containing signals for buy and sell actions.
  - `Sell` - Binary indicator for sell signals.
  - `Index` - Index of the data points.
  - `Buy` - Binary indicator for buy signals.

## Mathematical Formulation

### Step 1: Extract Relevant Columns as NumPy Array

The relevant columns are extracted from the DataFrame and converted to a NumPy array:

$$ \text{SellVector} = \begin{pmatrix} \text{Sell} & \text{Index} & \text{Buy} \end{pmatrix} $$

### Step 2: Identify Sell Signal Changes

Identify indices where the sell signal changes from 0 to 1 (indicating a new sell signal):

$$ a = \{ i \mid \text{Sell}_i - \text{Sell}_{i-1} = 1 \} $$

If the length of \( a \) is odd, the last element is removed to ensure even pairing:

$$ \text{if } |a| \% 2 \neq 0, \text{ then remove the last element of } a $$

### Step 3: Split Indices for Sell Dates

The indices are split into two equal parts:

$$ \text{Split1\_SellDatesVector} = \{ a_{2i} \} $$
$$ \text{Split2\_SellDatesVector} = \{ a_{2i+1} \} $$

### Step 4: Initialize Buy and Sell Date Arrays

Initialize arrays to store buy and sell dates:

$$ \text{buydatearray} = [0] $$
$$ \text{selldatearray} = [0] $$

### Step 5: Loop Through Split Indices and Extract Dates

Loop through each split vector to extract buy and sell dates:

For each pair in `Split1_SellDatesVector`:

1. Split `SellVector` at the specified indices.
2. Remove the first and second sub-arrays resulting from the split.
3. Check if there are any buy signals in the remaining sub-array.
4. If a buy signal is found, record the buy and sell dates:
   - `buydate`: First occurrence of a buy signal.
   - `selldate`: Last occurrence of the sub-array.

$$ \text{buydate} = \min \{ i \mid \text{Buy}_i = 1 \} + 2 $$
$$ \text{selldate} = \max \{ i \mid \text{Sell}_i = 1 \} + 1 $$

For each pair in `Split2_SellDatesVector`:

1. Repeat the above steps with a different delay for the buy date:

$$ \text{buydate} = \min \{ i \mid \text{Buy}_i = 1 \} + 1 $$

### Step 6: Append Extracted Dates to Arrays

Append the extracted buy and sell dates to the respective arrays:

$$ \text{buydate\_array} \leftarrow \text{buydate\_array} \cup \text{buydate} $$
$$\text{selldate\_array} \leftarrow \text{selldate\_array} \cup \text{selldate} $$
\\
$$ \text{buydate\_array} \leftarrow \begin{bmatrix} \text{buydate\_array} \\ \text{buydate} \end{bmatrix}$$
$$ \text{selldate\_array} \leftarrow \begin{bmatrix} \text{selldate\_array} \\ \text{selldate} \end{bmatrix} $$

Where:
$$\text{buydate\_array} = \begin{bmatrix} \text{date}_1 \\ \text{date}_2 \\ \vdots \\ \text{date}_n \end{bmatrix} $$
\\
$$ \text{selldate\_array} = \begin{bmatrix} \text{date}_1 \\ \text{date}_2 \\ \vdots \\ \text{date}_n \end{bmatrix} $$

### Outputs

- `buydate_array`: Array of buy dates after processing all signals.
- `selldate_array`: Array of sell dates after processing all signals.
- `volume_signals`: Original DataFrame containing volume signals.

The function returns the arrays of buy and sell dates, as well as the original volume signals DataFrame.



In [ ]:
SellVector=volume_signals[['Sell', 'Index','Buy']].to_numpy()
    #print(volume_signals[['Buy','Sell']])

    a=np.array(np.where(volume_signals['Sell'].diff()==1))

    if len(a[0])%2!=0:
        a=np.delete(a[0],-1)


    try:
        Split1_SellDatesVector=np.split(a,len(a)/2)
        Split2_SellDatesVector=np.split(a[1:-1],((len(a)-2))/2)
    except:
        Split1_SellDatesVector=np.split(a[0],len(a[0])/2)
        Split2_SellDatesVector=np.split(a[0][1:-1],((len(a[0])-2))/2)
    #print(Split2_SellDatesVector)
    #print(Split1_SellDatesVector)
    #Blir faktiskt ett litet fel här men spelar ingen roll i det långa loppet om en är fel.
    buydate_array=np.array([0])
    selldate_array=np.array([0])

    for x in range(len(Split1_SellDatesVector)):

        Splitted_Sell_Dates1=np.split(SellVector,Split1_SellDatesVector[x])
        Splitted_Sell_Dates1.pop(0)
        Splitted_Sell_Dates1.pop(1)

        two_axis=np.delete(Splitted_Sell_Dates1[0],0,axis=1)

        test=np.sum(two_axis, axis=0)
        if test[1]!=0:
            buynumber=np.where(two_axis[:,1]==1)[0][0]
            buydate=two_axis[buynumber][0]
            selldate=two_axis[-1][0]
            a=np.array([buydate])
            b=np.array([selldate])
            a += 2 # Ändra delay här! Ändra både a och b
            selldate_array=np.append(selldate_array,b+1, axis=0)
            buydate_array=np.append(buydate_array,a,axis=0)

        try:
            Splitted_Sell_Dates2=np.split(SellVector,Split2_SellDatesVector[x])
            Splitted_Sell_Dates2.pop(0)
            Splitted_Sell_Dates2.pop(1)
            two_axis2=np.delete(Splitted_Sell_Dates2[0],0,axis=1)
            test=np.sum(two_axis2,axis=0)

            if test[1]!=0:

                buynumber=np.where(two_axis2[:,1]==1)[0][0]
                buydate=two_axis2[buynumber][0]
                selldate=two_axis2[-1][0]
                a=np.array([buydate])
                b=np.array([selldate])
                #print(a)
                a += 1 # Ändra delay här! Ändra både a och b
                #print(a)
                selldate_array=np.append(selldate_array,b+1, axis=0)
                buydate_array=np.append(buydate_array,a,axis=0)
        except:pass

    #print(buydate_array,selldate_array)
    print(time.time()-start_time)
    return buydate_array,selldate_array,volume_signals

# **Drawdown**

This drawdown function simply uses the cummax-function to detect peaks. Pretty short and simple.



## Function Inputs and Parameters

- `portfolios`: Series or DataFrame representing the portfolio values over time.
- `df`: DataFrame containing the closing prices of the asset.
  - `Close` - Closing prices of the asset.

## Mathematical Formulation

### Step 1: Calculate Previous Peaks for Portfolio

The previous peaks in the portfolio values are calculated using the cumulative maximum function:

$$ \text{previous\_peaks}_{\text{portfolio}, i} = \max_{j \leq i} \text{portfolios}_j $$

### Step 2: Calculate Portfolio Drawdown

The drawdown for the portfolio is calculated as the relative difference between the current portfolio value and the previous peak, normalized by the previous peak:

$$ \text{drawdown}_{\text{portfolio}, i} = \frac{\text{portfolios}_i - \text{previous\_peaks}_{\text{portfolio}, i}}{\text{previous\_peaks}_{\text{portfolio}, i}} $$



### Step 3: Calculate Previous Peaks for Asset

The previous peaks in the asset's closing prices are calculated similarly:

$$\text{previous\_peaks}_{\text{asset}, i} = \max_{j \leq i} \text{Close}_j $$

### Step 4: Calculate Asset Drawdown

The drawdown for the asset is calculated as the relative difference between the current closing price and the previous peak, normalized by the previous peak:

$$ \text{drawdown}_{\text{asset}, i} = \frac{\text{Close}_i - \text{previous\_peaks}_{\text{asset}, i}}{\text{previous\_peaks}_{\text{asset}, i}} $$


### Outputs

The function prints the minimum drawdown values for both the portfolio and the asset. There are no returned values from this function.



In [ ]:
def drawdown(portfolios,df):
    previous_peaks = portfolios.cummax()

    drawdown = (portfolios - previous_peaks)/previous_peaks

    print('---'*20)
    print(f'{100*drawdown.min()}%')
    print('---'*20)

    previous_peaks2 = df['Close'].cummax()

    drawdown2 = (df['Close'] - previous_peaks2)/previous_peaks2

    print('---'*20)
    print(f'{100*drawdown2.min()}%')
    print('---'*20)